In [1]:
import os, sys
sys.path.append('../src')
from copulae1 import *
from KDEs import *
from toolbox import *
import json
import datetime
import pickle

import matplotlib.pyplot as plt

In [2]:
# Gaussian, t_Copula, Clayton, Frank, Gumbel, Plackett, Gaussian mix Indep
C1  = Gaussian(dict(rho=0.9),       Law_RS=stats.norm, Law_RF=stats.norm)
C2  = t_Copula(dict(rho=0.1, nu=4), Law_RS=stats.norm, Law_RF=stats.norm, nu_lowerbound=2)
C2c = t_Copula(dict(rho=0.1, nu=4), Law_RS=stats.norm, Law_RF=stats.norm, nu_lowerbound=4)
C3  = Clayton(dict(theta=0.1),      Law_RS=stats.norm, Law_RF=stats.norm)
C4  = Frank(dict(theta=0.1),        Law_RS=stats.norm, Law_RF=stats.norm)
C5  = Gumbel(dict(theta=3),         Law_RS=stats.norm, Law_RF=stats.norm)
C6  = Plackett(dict(theta=10),      Law_RS=stats.norm, Law_RF=stats.norm)
C7  = Gaussian_Mix_Independent(dict(rho=.5,p=0.7),Law_RS=stats.norm, Law_RF=stats.norm)
C8  = rot180Gumbel(dict(theta=3), Law_RS=stats.norm, Law_RF=stats.norm)

In [3]:
calibration_method = 'MM'
q_arr = [0.05,0.1,0.9,0.95]

In [4]:
Copulae_names = ['Gaussian', 't_Copula', 't_Copula_Capped', 'Clayton', 'Frank', 'Gumbel', 'Plackett', 'Gauss Mix Indep', 'rotGumbel']
Copulae_arr = [C1, C2, C2c, C3, C4, C5, C6, C7, C8]
Copulae = dict(zip(Copulae_names, Copulae_arr))

In [5]:
paras_results_dict = dict()
likelihood_results_dict = dict()

In [6]:
spot_name = 'rs'
future_name = 'rf'
data_name = 'ETHUSD_BTCUSD_25SEP20'
data_path = '/Volumes/external_SSD/copulaData/Deribit_ready2/%s/'%data_name

ls = os.listdir(data_path + 'train/')
ls = [l for l in ls if l.endswith('.csv')]

In [8]:
start = 0

for i, file in enumerate(ls[start:]):
	print('-----', i, len(ls), file, str(datetime.datetime.now()), '-----')
	# Calibration
	train = pd.read_csv(data_path + 'train/' + file)
	spot = train.loc[:, spot_name]
	future = train.loc[:, future_name]
	u = ECDF(spot)(spot)
	v = ECDF(future)(future)

	kde_brr = KDE(spot, "Gaussian")
	kde_btc = KDE(future, "Gaussian")

	for C_name in Copulae:
		Copulae[C_name].Law_RS = kde_brr
		Copulae[C_name].Law_RF = kde_btc

	paras = []
	likelihood = []
	for C_name in Copulae:
		if calibration_method == "MLE":
			Copulae[C_name].canonical_calibrate(u, v)

		elif calibration_method == "MM":
			Copulae[C_name].mm_calibrate(u, v, q_arr)

		print(C_name, 'is done.\n')

	for C_name in Copulae:
		paras.append((C_name, Copulae[C_name].paras))

	for C_name in Copulae:
		ln = Copulae[C_name].dependency_likelihood(u, v)
		likelihood.append((C_name, ln))
        
	paras_results_dict[file] = paras
	likelihood_results_dict[file] = likelihood

	if (i%60==0) or (i == 0) or (i==len(ls)):
		print('saved!')
		with open(data_name+'_paras.pickle', 'wb') as f:
			pickle.dump(paras_results_dict, f)

		with open(data_name+'_likelihood.pickle', 'wb') as f:
			pickle.dump(likelihood_results_dict, f)

----- 0 186 0.csv 2022-10-12 11:48:15.896390 -----


/Users/francis/Google Drive/00_IRTG1792/copula/revision/copulae1.py:167: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  m_hat = np.array([self.tau()] + [self._lambda(q) for q in q_arr])


Optimization terminated successfully.
         Current function value: 0.021683
         Iterations: 31
         Function evaluations: 65
Gaussian is done.

Optimization terminated successfully.
         Current function value: 0.020949
         Iterations: 134
         Function evaluations: 284
t_Copula is done.

Optimization terminated successfully.
         Current function value: 0.020949
         Iterations: 111
         Function evaluations: 233
t_Copula_Capped is done.

Optimization terminated successfully.
         Current function value: 0.182365
         Iterations: 45
         Function evaluations: 99
Clayton is done.

Optimization terminated successfully.
         Current function value: 0.068118
         Iterations: 50
         Function evaluations: 112
Frank is done.

Optimization terminated successfully.
         Current function value: 0.159165
         Iterations: 33
         Function evaluations: 75
Gumbel is done.

Optimization terminated successfully.
         Curre

/Users/francis/Google Drive/00_IRTG1792/copula/revision/toolbox.py:55: RuntimeWarning: overflow encountered in double_scalars
  fn = lambda s: s ** (self.nu - 1) * np.exp(-s ** 2 / 2) * self.MN.cdf(s * b / np.sqrt(self.nu))
/Users/francis/Google Drive/00_IRTG1792/copula/revision/toolbox.py:55: RuntimeWarning: invalid value encountered in double_scalars
  fn = lambda s: s ** (self.nu - 1) * np.exp(-s ** 2 / 2) * self.MN.cdf(s * b / np.sqrt(self.nu))
/Users/francis/Google Drive/00_IRTG1792/copula/revision/toolbox.py:56: IntegrationWarning: The occurrence of roundoff error is detected, which prevents 
  the requested tolerance from being achieved.  The error may be 
  underestimated.
  return 2 ** (1 - (self.nu / 2)) / gamma(self.nu / 2) * scipy.integrate.quad(fn, 0, np.inf)[0]


Optimization terminated successfully.
         Current function value: 0.037379
         Iterations: 155
         Function evaluations: 305
t_Copula is done.

Optimization terminated successfully.
         Current function value: 0.037379
         Iterations: 152
         Function evaluations: 297
t_Copula_Capped is done.

Optimization terminated successfully.
         Current function value: 0.173780
         Iterations: 39
         Function evaluations: 87
Clayton is done.

Optimization terminated successfully.
         Current function value: 0.055956
         Iterations: 41
         Function evaluations: 93
Frank is done.

Optimization terminated successfully.
         Current function value: 0.198360
         Iterations: 35
         Function evaluations: 76
Gumbel is done.

Optimization terminated successfully.
         Current function value: 0.033302
         Iterations: 39
         Function evaluations: 87
Plackett is done.

Optimization terminated successfully.
         Curren

...

Optimization terminated successfully.
         Current function value: 0.029285
         Iterations: 67
         Function evaluations: 142
Gauss Mix Indep is done.

Optimization terminated successfully.
         Current function value: 0.035351
         Iterations: 32
         Function evaluations: 70
rotGumbel is done.

----- 158 186 74.csv 2022-10-12 12:49:15.547127 -----
Optimization terminated successfully.
         Current function value: 0.031164
         Iterations: 30
         Function evaluations: 62
Gaussian is done.

Optimization terminated successfully.
         Current function value: 0.029833
         Iterations: 72
         Function evaluations: 158
t_Copula is done.

Optimization terminated successfully.
         Current function value: 0.029833
         Iterations: 70
         Function evaluations: 155
t_Copula_Capped is done.

Optimization terminated successfully.
         Current function value: 0.216216
         Iterations: 32
         Function evaluations: 71
Clayt

Optimization terminated successfully.
         Current function value: 0.047062
         Iterations: 65
         Function evaluations: 134
Gauss Mix Indep is done.

Optimization terminated successfully.
         Current function value: 0.015172
         Iterations: 32
         Function evaluations: 68
rotGumbel is done.

----- 164 186 8.csv 2022-10-12 12:51:01.973713 -----
Optimization terminated successfully.
         Current function value: 0.021309
         Iterations: 30
         Function evaluations: 65
Gaussian is done.

Optimization terminated successfully.
         Current function value: 0.021356
         Iterations: 125
         Function evaluations: 247
t_Copula is done.

Optimization terminated successfully.
         Current function value: 0.021356
         Iterations: 144
         Function evaluations: 270
t_Copula_Capped is done.

Optimization terminated successfully.
         Current function value: 0.198011
         Iterations: 36
         Function evaluations: 82
Clay

Optimization terminated successfully.
         Current function value: 0.047032
         Iterations: 67
         Function evaluations: 142
Gauss Mix Indep is done.

Optimization terminated successfully.
         Current function value: 0.015098
         Iterations: 33
         Function evaluations: 72
rotGumbel is done.

----- 170 186 85.csv 2022-10-12 12:53:57.020522 -----
Optimization terminated successfully.
         Current function value: 0.047786
         Iterations: 31
         Function evaluations: 65
Gaussian is done.

Optimization terminated successfully.
         Current function value: 0.047455
         Iterations: 80
         Function evaluations: 181
t_Copula is done.

Optimization terminated successfully.
         Current function value: 0.047455
         Iterations: 79
         Function evaluations: 181
t_Copula_Capped is done.

Optimization terminated successfully.
         Current function value: 0.148025
         Iterations: 32
         Function evaluations: 70
Clayt

Optimization terminated successfully.
         Current function value: 0.021274
         Iterations: 71
         Function evaluations: 146
Gauss Mix Indep is done.

Optimization terminated successfully.
         Current function value: 0.025747
         Iterations: 34
         Function evaluations: 74
rotGumbel is done.

----- 176 186 90.csv 2022-10-12 12:55:50.673865 -----
Optimization terminated successfully.
         Current function value: 0.020900
         Iterations: 31
         Function evaluations: 65
Gaussian is done.

Optimization terminated successfully.
         Current function value: 0.016577
         Iterations: 110
         Function evaluations: 230
t_Copula is done.

Optimization terminated successfully.
         Current function value: 0.016718
         Iterations: 190
         Function evaluations: 372
t_Copula_Capped is done.

Optimization terminated successfully.
         Current function value: 0.217257
         Iterations: 33
         Function evaluations: 74
Cla

Optimization terminated successfully.
         Current function value: 0.020525
         Iterations: 71
         Function evaluations: 149
Gauss Mix Indep is done.

Optimization terminated successfully.
         Current function value: 0.021200
         Iterations: 36
         Function evaluations: 77
rotGumbel is done.

----- 182 186 96.csv 2022-10-12 12:57:13.541856 -----
Optimization terminated successfully.
         Current function value: 0.030232
         Iterations: 30
         Function evaluations: 63
Gaussian is done.

Optimization terminated successfully.
         Current function value: 0.013381
         Iterations: 133
         Function evaluations: 258
t_Copula is done.

Optimization terminated successfully.
         Current function value: 0.019199
         Iterations: 101
         Function evaluations: 198
t_Copula_Capped is done.

Optimization terminated successfully.
         Current function value: 0.230128
         Iterations: 35
         Function evaluations: 78
Cla

In [14]:
with open(data_name+'_paras.pickle', 'wb') as f:
    pickle.dump(paras_results_dict, f)

with open(data_name+'_likelihood.pickle', 'wb') as f:
    pickle.dump(likelihood_results_dict, f)